import necessary libraries

In [4]:
from selenium import webdriver
import time
import argparse

In [341]:
class Scraper():

    #use selenium to open desired webpage using Chrome
    def __init__(self, url):
        self.driver = webdriver.Chrome()
        self.url = url
        self.page = self.driver.get(self.url)

        #main book categories of the website
        self.book_main_categories = []
        self.category_subcategories = []
        self.books_list_page = []
        self.books_list_slider = []
    
    #bypass cookies
    def bypass_cookies(self):
    
        # wait so website doesn't suspect you are a bot
        time.sleep(2)

        # if website has cookies
        try:
            accept_cookies_button = self.driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
            accept_cookies_button.click()

        # if website doesn't have cookies 
        except: 
            pass

    # define flags for different conditions under which you can run the scraper
    def scraper_flags(self):
        # first flag is for choosing desired book category
        self.category_parser = argparse.ArgumentParser(description = "Which main book category you want to select?")

        ##for when runing .py file!!
        # f for fiction
        self.category_parser.add_argument('-f', action = "store_true")
        # c for crime
        self.category_parser.add_argument('-c', action = "store_true")
        # sf for science-fiction
        self.category_parser.add_argument('-sf', action = "store_true")
        # g for graphic novels and manga
        self.category_parser.add_argument('-g', action = "store_true")
        # nf for non-fiction
        self.category_parser.add_argument('-nf', action = "store_true")

        ## for now in jupyter notebook use the below
        #self.category_parser.add_argument(category_flag, action = "store_true")

        self.category_args = self.category_parser.parse_args()

    
    #click/access one of the links
    def access_book_category(self, item):
        self.driver.get(item)  

    #retrieves list of link with main books categories
    def get_main_books_categories(self, desired_category):

        #first part of xpath always navigated to desktop version of website desktop-navs
        book_category_path = self.driver.find_element_by_xpath('//div[@class = "navs-container desktop-navs"]/ul[@class = "subnavs"][1]/li[2]')
        # get all a elements containing the book categories
        books_main_list = book_category_path.find_elements_by_xpath('.//span[@class = "name nav-header-link"]/a')
        # we only interested in 5 categories: fiction, crime, science finction, graphic novel and non-fiction
        # leaves out the children book categories
        books_main_list = books_main_list[0:5]

        # make this into a list compression 
        for item in books_main_list:
            #a_tag = item.find_element_by_tag_name('a')
            link = item.get_attribute('href')
            self.book_main_categories.append(link)

        #return book_main_categories
        #based on passed category_flag, return desired link for next method
        ## FOR WHEN RUNING .PY SCRIPT
        #if self.category_args.fiction == True:
         #   return self.book_main_categories[0]
        #elif self.category_args.c == True:
         #   return self.book_main_categories[1]
        #elif self.category_args.sf == True:
         #   return self.book_main_categories[2]
        #elif self.category_args.g == True:
         #   return self.book_main_categories[3]
        #elif self.category_args.nf == True:
         #   return self.book_main_categories[4]

         ## JUST FOR NOW FOR IPYNB FILES
        if desired_category == 'fiction':
            return self.book_main_categories[0]
        elif desired_category == 'crime':
            return self.book_main_categories[1]
        elif desired_category == 'science fiction':
            return self.book_main_categories[2]
        elif desired_category == 'graphic novel':
            return self.book_main_categories[3]
        elif desired_category == 'non fiction':
            return self.book_main_categories[4]



    #from main category - can now access all sub-categories buttons
    def get_category_subcategories(self, desired_category):

        self.category_subcategories = []
        # calls function to retrieve desired book category and then loads corresponding page
        category_header = self.get_main_books_categories(desired_category)
        self.access_book_category(category_header)

        #find list of category sub-divisions
        subcategories_list = self.driver.find_elements_by_xpath('//div[@class = "span3 tablet-span6 mobile-span6"]//a')

        for item in subcategories_list:
            link = item.get_attribute('href')
            self.category_subcategories.append(link)

    
    ## subcategory page: check if 'Our best <subcategory> exist if so you can scroll left/right and press
    ## see all to display all the pages, otherwise skips this steps and directly goes to all pages display
    
    def finds_best_subcategory_list(self):

        #for subcategory in self.category_subcategories :

        # if header and see more button then press this - get data from full pages
        
        header_path = self.driver.find_element_by_xpath('//header[@class = "span12 pages-header-row"]')

            #find see all button - sometimes called see more but same xpath
        self.see_all_button = header_path.find_element_by_xpath('./a[@class = "button button-teal"]')
        self.see_all_button.click()
        time.sleep(3)

        try:
            header_path = self.driver.find_element_by_xpath('//header[@class = "span12 pages-header-row"]')
            self.see_all_button = header_path.find_element_by_xpath('./a[@class = "button button-teal"]')
            self.see_all_button.click()
            time.sleep(3)
        except:
            pass

        #self.get_book_list_page(3)
        
        #if you want instead to only get information from slider
        #self.get_book_list_slider()
                        
        
    ##GET FINAL BOOKS LIST
    
    #navigation method 1: get current url, scroll down, get next page url - FASTEST METHOD
    #returns book list from see more pages
    def get_book_list_page(self, pages_no):

        if self.driver.current_url[-7 : -1] == "?page=":

            self.current_url = self.driver.current_url[:-7] + '/page/'

            i = 1
            while i <= pages_no:

                self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                list_books_links = self.driver.find_elements_by_xpath('//div[@class = "image-wrap"]/a')
                for item in list_books_links:
                    link = item.get_attribute('href')
                    self.books_list_page.append(link)

                i += 1
                self_new_current_url = ''
                self.new_current_url = self.current_url + str(i)
                self.driver.get(self.new_current_url) 

        
        elif self.driver.current_url[-7 : -1] == "/page/":

            self.current_url = self.driver.current_url[:-1]

            i = 1
            while i <= pages_no:

                self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                list_books_links = self.driver.find_elements_by_xpath('//div[@class = "image-wrap"]/a')
                for item in list_books_links:
                    link = item.get_attribute('href')
                    self.books_list_page.append(link)

                i += 1
                self_new_current_url = ''
                self.new_current_url = self.current_url + str(i)
                self.driver.get(self.new_current_url)

        else:

            self.current_url = self.driver.current_url + '/page/'
            i = 1
            while i <= pages_no:

                self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                list_books_links = self.driver.find_elements_by_xpath('//div[@class = "image-wrap"]/a')
                for item in list_books_links:
                    link = item.get_attribute('href')
                    self.books_list_page.append(link)

                i += 1
                self_new_current_url = ''
                self.new_current_url = self.current_url + str(i)
                self.driver.get(self.new_current_url)


    
    ## loop through book links, access one at a time and then collect the meta-data from each book page
    ## first method: enter book and navigate to click & collect information
    # will need to adapt it to loop through all books
    def click_and_collect(self, index):
        self.access_book_category(self.all_links_next_page, index)

        click_and_collect_button = self.driver.find_element_by_xpath('//div[@class = "book-actions"]//button[@class = "button button-gold js-open-modal"]')
        click_and_collect_button.click()

        search_bar = self.driver.find_element_by_xpath('//input[@placeholder = "Town, city, or postcode"]')
        time.sleep(5)
        search_bar.send_keys("WC1 0RW")

        go_button = self.driver.find_element_by_xpath('//button[@id = "searchterm"]')
        go_button.click()



    

#### also removes duplicate links

In [ ]:
from collections import Iterable
def flatten(lis):
     for item in lis:
         if isinstance(item, Iterable) and not isinstance(item, str):
             for x in flatten(item):
                 yield x
         else:        
             yield item

#### if name == main run

In [355]:
if __name__ == "__main__":

    waterstones = Scraper("https://www.waterstones.com")
    waterstones.bypass_cookies()

    list_categories = ["fiction", "crime", "science fiction", "graphic novel", "non fiction"]
    final_book_list = []


    for category in list_categories:

        waterstones.get_category_subcategories(category)

        for subcategory in waterstones.category_subcategories:

            waterstones.access_book_category(subcategory)
            waterstones.finds_best_subcategory_list()
            waterstones.get_book_list_page(3)
            final_book_list.append(waterstones.books_list_page)
    
    currated_list = list(flatten(final_book_list))


In [356]:
len(list(flatten(final_book_list)))

41472

Initiate Scraper object with desired url

In [342]:
waterstones = Scraper("https://www.waterstones.com")

In [343]:
waterstones.bypass_cookies()

In [344]:
waterstones.get_category_subcategories('non fiction')
waterstones.category_subcategories

['https://www.waterstones.com/category/biography-true-stories',
 'https://www.waterstones.com/category/food-drink',
 'https://www.waterstones.com/category/history',
 'https://www.waterstones.com/category/sports-leisure',
 'https://www.waterstones.com/category/art-fashion-photography',
 'https://www.waterstones.com/category/health-lifestyle',
 'https://www.waterstones.com/category/popular-science-nature/popular-science',
 'https://www.waterstones.com/category/travel-maps']

In [318]:
waterstones.category_subcategories[7]

'https://www.waterstones.com/category/travel-maps'

#### navigation method 1

In [114]:
waterstones.scroll_see_more_navigate(3)

#### navigation method 2

In [115]:
waterstones.scroll_down_up_next(3)

#### navigation method 3

In [116]:
waterstones.navigation_method_3(3)

extra code from initial scraper

In [117]:
   #method 1: scroll down to end of page, wait, press load more and so on (number rep = number of pages scrolled)
    #inside this method later will need to also get list of all links to the books -SECOND PLACE SPEED
    def navigation_method_1(self, pages_no):

        scroll_pause_time = 2
        #gets current page height
        last_height = self.driver.execute_script("return document.body.scrollHeight")
        
        #here using while true loop it go on forever - for now add counter to only load first 5 pages
        #some problem with page number!! only loads until page 4 but no further?
        i = 0
        while i <= pages_no:
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(scroll_pause_time)

            new_height = self.driver.execute_script("return document.body.scrollHeight")

            if new_height == last_height:

                all_books = self.driver.find_elements_by_xpath('//div[@class = "image-wrap"]/a')
                for item in all_books:
                    link = item.get_attribute('href')
                    self.all_links_page.append(link)
                show_more_button = self.driver.find_element_by_xpath('//button[@class = "button button-teal"]')
                show_more_button.click()
                break

            #once at the bottom of page and see more pressed- reinitialises height
            last_height = new_height
            i +=1
    
    #method 2: scroll down, scroll back up and then press next arrow to navigate to next page -SLOWEST
    def navigation_method_2(self, pages_no):

        scroll_pause_time = 2

        i = 1
        while i <= pages_no:
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            all_books_this_page = self.driver.find_elements_by_xpath('//div[@class = "image-wrap"]/a')
            for item in all_books_this_page:
                link = item.get_attribute('href')
                self.all_links_next_page.append(link)

            #time.sleep(scroll_pause_time)

            try:
                go_back_up_button = self.driver.find_element_by_xpath('//div[@class = "backtotop show stick"]')
                go_back_up_button.click()
            except:
                go_back_up_button = self.driver.find_element_by_xpath('//div[@class = "backtotop show"]')
                go_back_up_button.click()

            time.sleep(scroll_pause_time)

            next_page = self.driver.find_element_by_xpath('//a[@title = "Go to next page"]')
            next_page.click()
            i += 1
            #break

IndentationError: unexpected indent (2578035342.py, line 3)

In [ ]:

    #alternatively can get directly a link to all existent sub-categories of all main categories
    def get_all_subcategories(self):
        all_subcategories = self.driver.find_elements_by_xpath('//div[@class = "navs-container desktop-navs"]//li[@class = "subnav"][2]//li[@class = "subnav-item"]')

        for item in all_subcategories:
            a_tag = item.find_element_by_tag_name('a')
            link = a_tag.get_attribute('href')
            self.all_subcategories.append(link)

In [ ]:
  
    # method 2 returning book list from slider bar
    def get_book_list_slider(self):

        swiper_path = self.driver.find_element_by_xpath('//div[@class = "swiper-wrapper"]')
        titles_path = swiper_path.find_elements_by_xpath('.//div[@class = "title-wrap"]')

        for item in titles_path:
            a_tag = item.find_element_by_tag_name('a')
            link = a_tag.get_attribute('href')
            self.books_list_slider.append(link)